In [70]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [71]:
'''
STEP 1: LOADING DATASET
'''

transform=transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])
trainset=datasets.MNIST('~/.pytorch/MNIST_data/',train=True,transform=transform,download=True)
testset=datasets.MNIST('~/.pytorch/MNIST_data/',train=False,transform=transform,download=True)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=98,shuffle=True,num_workers=0)
#will explain later
testloader=torch.utils.data.DataLoader(testset,batch_size=98,shuffle=True,num_workers=0)


In [72]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 90)
        self.fc2 = nn.Linear(90, 30)
        self.fc3 = nn.Linear(30, 10)

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc3(x), dim=1)

        return x
        
model=Network()
#optimizer=optim.Adam(model.parameters(),lr=0.001,momentum = 0.9)
optimizer = optim.SGD(model.parameters(), lr=0.008, momentum=0.9, weight_decay=3.0e-4, nesterov = True)
criterion=nn.NLLLoss()

In [73]:
## Train

epochs=10
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in trainloader:
        optimizer.zero_grad()
        log_ps=model(images)
        loss=criterion(log_ps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels)
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor))
        model.train()
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Total no. of parameters : ",pytorch_total_params )        
        

Epoch: 1/10..  Training Loss: 0.772..  Test Loss: 0.306..  Test Accuracy: 0.909
Epoch: 2/10..  Training Loss: 0.366..  Test Loss: 0.228..  Test Accuracy: 0.930
Epoch: 3/10..  Training Loss: 0.294..  Test Loss: 0.180..  Test Accuracy: 0.949
Epoch: 4/10..  Training Loss: 0.256..  Test Loss: 0.153..  Test Accuracy: 0.954
Epoch: 5/10..  Training Loss: 0.233..  Test Loss: 0.147..  Test Accuracy: 0.954
Epoch: 6/10..  Training Loss: 0.215..  Test Loss: 0.130..  Test Accuracy: 0.960
Epoch: 7/10..  Training Loss: 0.197..  Test Loss: 0.135..  Test Accuracy: 0.958
Epoch: 8/10..  Training Loss: 0.191..  Test Loss: 0.120..  Test Accuracy: 0.963
Epoch: 9/10..  Training Loss: 0.178..  Test Loss: 0.112..  Test Accuracy: 0.967
Epoch: 10/10..  Training Loss: 0.170..  Test Loss: 0.109..  Test Accuracy: 0.968
Total no. of parameters :  73690
